In [15]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np 
import os 
import json 
import torch 

sns.set_theme(style="whitegrid")
sns.set_context("paper", font_scale=1.5)


In [38]:
exp_name = "exp_1_not"
dir_path = f"results/{exp_name}/General_Binary"
models = [
          'openai--gpt-4o-mini', 
          'llama3_1_instruct--70b', 
          'llama3_1--8b', 
          'gemma2--9b', 
        #   'exaone--8b', 
          'qwen2--7b'
          ]
result_df = {
    'model': [],
    'diff' : [],  # strong - none 
    'mv' : []
}
values = {
}
strong_index = 0 
none_index = 0 

def process_model_output(x):
    x =str(x)
    if "1" in x and "0" in x:
        return 2
    elif "1" in x:
        return 1
    elif "0" in x:
        return 0
    else:
        return -1 

def mv_func(x):
    values = ['must', 'should', 'ought']
    for v in values:
        if v in x:
            return v 
    if "has to" in x or "have to" in x:
        return "have to"
    return "none"

for model in models:
    result_df = pd.read_csv(os.path.join(dir_path, f"{model}.csv"))
    for line in result_df.iterrows():
        line = line[1]
        mv = mv_func(line['input'])
        if line['input_type'] == 'strong':
            if strong_index not in values:
                values[strong_index] = {}
            values[strong_index]['strong'] = {'value':process_model_output(line['model_output']), 
                                           'mv': mv,
                                           'input': line['input'],
                                           'model': model}
            strong_index += 1
        elif line['input_type'] == 'none':
            if none_index not in values:
                values[none_index] = {}
            values[none_index]['none'] = {'value':process_model_output(line['model_output']), 
                                           'mv': mv,
                                           'input': line['input'],
                                           'model': model}
            none_index += 1 

In [52]:
result_df = {
    'model': [],
    'diff' : [],  # strong - none 
    'strong': [],
    'none': [],
    'mv' : [],
    # 'sample_index':[],
}
for i in range(len(values)):
    result_df['model'].append(values[i]['strong']['model'])
    result_df['diff'].append(values[i]['strong']['value'] - values[i]['none']['value'])
    result_df['strong'].append(values[i]['strong']['value'])
    result_df['none'].append(values[i]['none']['value'])
    result_df['mv'].append(values[i]['strong']['mv'])
    # result_df['sample_index'].append(i)
    
result_df = pd.DataFrame(result_df)
result_df

,model,diff,strong,none,mv
0,openai--gpt-4o-mini,1,1,0,should
1,openai--gpt-4o-mini,1,1,0,must
2,openai--gpt-4o-mini,1,1,0,ought
3,openai--gpt-4o-mini,0,0,0,must
4,openai--gpt-4o-mini,1,1,0,should
...,...,...,...,...,...
2220,qwen2--7b,1,1,0,must
2221,qwen2--7b,1,1,0,should
2222,qwen2--7b,1,1,0,should
2223,qwen2--7b,1,1,0,should


In [53]:
result_df.groupby(['model', 'mv']).mean()
result_df.groupby(['model', 'mv']).var()

diff    strong      none
model                  mv                                  
gemma2--9b             must    0.448276  0.866995  0.418719
                       ought   0.490000  0.920000  0.430000
                       should  0.338028  0.795775  0.457746
llama3_1--8b           must    0.019704  1.000000  0.980296
                       ought   0.010000  1.000000  0.990000
                       should  0.000000  1.000000  1.000000
llama3_1_instruct--70b must    0.674877  0.763547  0.088670
                       ought   0.700000  0.770000  0.070000
                       should  0.492958  0.556338  0.063380
openai--gpt-4o-mini    must    0.714286  0.852217  0.137931
                       ought   0.860000  0.970000  0.110000
                       should  0.746479  0.873239  0.126761
qwen2--7b              must    0.694581  0.916256  0.221675
                       ought   0.700000  0.820000  0.120000
                       should  0.492958  0.683099  0.190141